## spark read s3 delta

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Window, Row

from PyQt5.QtWidgets import QApplication
from PyQt5.QtWebEngineWidgets import QWebEngineView
import pandas as pd
from pyspark.sql import SparkSession
from iPython import display

import pprint
import boto3
import json
import os

pp = pprint.PrettyPrinter(indent = 3)
print('imported modules.')

In [ ]:
client = boto3.client('s3')

bucket_meta = client.list_objects(Bucket = 'yelp-dataset-stevenhurwitt')
print('files in s3 bucket:')
print('')
for c in bucket_meta['Contents']:
    print(c['Key'])

In [ ]:
def read_json(filename):
    """
    reads a yelp .json file from s3 bucket.

    keyword arguments:
    filename - name of file (str)

    returns: json_file (json)
    """

    response = client.get_object(Bucket = 'yelp-academic-dataset-stevenhurwitt', Key = filename)
    file_content = response['Body'].read().decode('utf-8')
    json_file = json.loads("[" + file_content.replace("}\n{", "},\n{") + "]")
    return(json_file)

In [ ]:
dynamodb = boto3.resource('dynamodb', endpoint_url="https://us-east-2.console.aws.amazon.com/dynamodbv2/home?region=us-east-2#export?arn=arn:aws:dynamodb:us-east-2134132211607:8000")

In [ ]:
yelp_business = dynamodb.create_table(
        TableName='yelp.business',
        KeySchema=[
            {
                'AttributeName': 'business_id',
                'KeyType': 'HASH'  # Partition key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'name',
                'AttributeType': 'S'
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 25,
            'WriteCapacityUnits': 20
        }
    )
print('created dynamo table.')

In [ ]:
business = read_json('s3a://yelp-dataset-stevenhurwitt/yelp_academic_dataset_business.json')
business.head()
print('read in json file.')

In [ ]:
print('json file has {} records with size of {} mb.'.format(len(business), sys.getsizeof(business)/1000000))
print('here is an example record:')
print('')
pp.pprint(business[0])

In [ ]:
df_pandas = business.toPandas()

In [ ]:
html_df = df_pandas.to_html()

In [ ]:
display(html_df)

In [ ]:
# ! gcloud auth login --no-launch-browser

In [ ]:
try:
    response = table.query(
        ProjectionExpression="user, created_at",
        ExpressionAttributeNames={"user": "created_at"},
        KeyConditionExpression=(
            Key('user').eq("ihearttyou2") &
            Key('created_at').between('2011-01-01-01T00:00:00Z', '2024-05-25T23:59:59Z')))
            
except ClientError as err:
    if err.response['Error']['Code'] == "ValidationException":
        logger.warning(
            "There's a validation error. Here's the message: %s: %s",
            err.response['Error']['Code'], err.response['Error']['Message'])
    else:
        logger.error(
            "Couldn't query for movies. Here's why: %s: %s",
            err.response['Error']['Code'], err.response['Error']['Message'])
        raise
else:
    response_itmes = response['Items']
    response_item_count = response.item_count
    logger.info(response_items.shape)

In [ ]:
response[0]